In [1]:
import polars as pl
from polars import col as c
import networkx as nx

from config import settings
import json
import os
from datetime import datetime, UTC
import datetime as dt

from utility.polars_operation import generate_uuid_col
from utility.parser_utility import (
    add_table_to_changes_schema,
    generate_connectivity_table,
    generate_random_uuid,
)
from utility.general_function import pl_to_dict

from twindigrid_changes.schema import ChangesSchema
from twindigrid_sql.schema.enum import (
    MeasurementClass,
    MeasurementPhase,
    MeasurementColumn,
    SubstationType,
    TerminalSide,
)
from twindigrid_sql.entries.source import (
    SCADA,
    CONVENTIONAL_METER,
    GRID_LOAD,
    SCADA,
    ESTIMATED,
)


from twindigrid_sql.entries.equipment_class import (
    TRANSFORMER,
    BRANCH,
    SWITCH,
    INDIRECT_FEEDER,
    BUSBAR_SECTION,
    ENERGY_CONSUMER,
)
from twindigrid_sql.entries.measurement_type import ENERGY, ACTIVE_POWER, REACTIVE_POWER
from twindigrid_sql.entries.unit_symbol import WATTHOUR, WATT

# Useless outside jupiternotebook because in settings.py a line that changes the directory to src for ipynb
os.chdir(os.getcwd().replace("/src", ""))
# os.getcwd()

# Import data from matlab

In [2]:
file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))

In [3]:
parameter_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_parameter"])
nodedata_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_node_data"])
powerdata_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_Power_data"])
linedata_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_Line_data"])
result_distflow: pl.DataFrame = pl.read_csv(file_names["Distflow_result"])
# nodedata_distflow = nodedata_distflow.with_columns(c("Snom").cast(pl.Int8))
# # To have base value (need lenght of line), not from matlab !
# line_data_from_input_file: pl.DataFrame = pl.read_excel(
#     file_names["Line_Data_From_Input_File"]
# )

# Add node number to power data
powerdata_distflow = powerdata_distflow.with_row_index(
    "node_number", offset=1
)  # offset=1 because slack bus is 0 and no power on it
powerdata_distflow = powerdata_distflow.with_columns(c("node_number").cast(pl.Int64))
# Create a topology dataframe with basic topology information

df_topology = nodedata_distflow.select(
    c("index").alias("node_number"),
    c("Vnom"),
)

# Add the power data to the topology dataframe with node as key
df_topology = df_topology.join(
    powerdata_distflow, on="node_number", how="full", coalesce=True
)
nodedata_distflow

index,indexLines_1,indexLines_2,indexLines_3,Vnom,Snom,powerFactor_1,powerFactor_2,Bnode,SM,Annual
i64,i64,i64,i64,i64,str,f64,i64,i64,f64,f64
0,1,2,3,400,null,0.8,1,0,NaN,0.0
1,4,5,null,400,null,0.8,1,0,NaN,0.0
2,6,7,8,400,null,0.8,1,0,NaN,0.0
3,9,null,null,400,null,0.8,1,0,NaN,0.0
4,null,null,null,400,null,0.8,1,0,12.0,0.0
…,…,…,…,…,…,…,…,…,…,…
53,null,null,null,400,null,0.8,1,0,NaN,3095.956989
54,55,56,null,400,null,0.8,1,0,NaN,0.0
55,57,null,null,400,null,0.8,1,0,NaN,0.0


# Set missing value for equipment

In [4]:
### Set missing value for equipment
# Fake value for the length of the branch
base_length = 1
# Fake value for the switch state
switch_state = False
switch_type = "locked_switch"
switch_command = "unknown"

# Initialisation
default_install_date: datetime = datetime(*settings.DEFAULT_INSTALL_DATE, tzinfo=UTC)
heartbeat = datetime.now(UTC)
changes_schema = ChangesSchema()

## Connectivity node table

In [5]:
# Generate the node dict with uuid for each node
connectivity_node: dict[float, str] = pl_to_dict(
    df_topology.select(
        c("node_number"),
        c("node_number").pipe(generate_uuid_col, added_string="node_").alias("uuid"),
    )
)
## Add the cn_fk to the topology dataframe
df_topology = df_topology.with_columns(
    c("node_number").replace_strict(connectivity_node, default=None).alias("cn_fk")
)

## Branch

In [6]:
# branch :pl.DataFrame =


# Current and other line parameter in pu

# Filter to take only branch, connection_type == 2
branch = linedata_distflow.filter(c("connection_type") == 2).select(
    c("line_number").alias("dso_code"),
    c("i_pu").alias("current_limit"),
    c("r_pu"),
    c("x_pu"),
    c("b_pu"),
    # Need column name non null value for validation of the schema
    pl.lit(base_length).alias("length"),  # km
    pl.lit(BRANCH).alias("concrete_class"),
    pl.lit(default_install_date).alias("start"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    c("line_number").pipe(generate_uuid_col, added_string=BRANCH).alias("uuid"),
    # Generate uuid for each terminal of branch with node uuid
    c("node_from").replace_strict(connectivity_node, default=None).alias("t1"),
    c("node_to").replace_strict(connectivity_node, default=None).alias("t2"),
    # Need column name for validation of the schema
    pl.lit(None).alias("t1_container_fk"),
    pl.lit(None).alias("t2_container_fk"),
)
branch

dso_code,current_limit,r_pu,x_pu,b_pu,length,concrete_class,start,start_heartbeat,uuid,t1,t2,t1_container_fk,t2_container_fk
i64,f64,f64,f64,f64,i32,str,"datetime[μs, UTC]","datetime[μs, UTC]",str,str,str,null,null
1,0.024942,0.19375,0.112969,7.9168e-8,1,"""branch""",1960-01-01 00:00:00 UTC,2025-01-13 06:58:59.266502 UTC,"""b4fdf239-cfe2-5453-91c3-84c1b2…","""ba84d70a-80d7-590e-b112-f9c4b5…","""df941fce-ceda-5874-ab63-5c8af9…",null,null
2,0.026327,2.046875,1.14625,8.9632e-7,1,"""branch""",1960-01-01 00:00:00 UTC,2025-01-13 06:58:59.266502 UTC,"""3edce6fc-e164-5b3e-9b0d-8a1c86…","""078656ed-79f8-53a1-a67a-bb8f53…","""df941fce-ceda-5874-ab63-5c8af9…",null,null
4,0.003048,3.03,0.0911,2.2696e-8,1,"""branch""",1960-01-01 00:00:00 UTC,2025-01-13 06:58:59.266502 UTC,"""c173d7f3-f7ca-5f44-98d8-63c1aa…","""af72457f-f983-5eeb-a635-0609f4…","""ba84d70a-80d7-590e-b112-f9c4b5…",null,null
5,0.024942,0.0775,0.0451875,3.1667e-8,1,"""branch""",1960-01-01 00:00:00 UTC,2025-01-13 06:58:59.266502 UTC,"""c33afba9-0379-5d6a-9ca1-743cf7…","""dbd2411e-1e87-5956-86d9-d69ee7…","""ba84d70a-80d7-590e-b112-f9c4b5…",null,null
6,0.026327,0.4140625,0.231875,1.8132e-7,1,"""branch""",1960-01-01 00:00:00 UTC,2025-01-13 06:58:59.266502 UTC,"""020bd95c-9320-5560-9627-4bbe19…","""2db84a3d-aa74-5b4f-866a-331318…","""078656ed-79f8-53a1-a67a-bb8f53…",null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
53,0.003048,3.7875,0.113875,2.8370e-8,1,"""branch""",1960-01-01 00:00:00 UTC,2025-01-13 06:58:59.266502 UTC,"""d9975d35-54c3-5993-b044-835e1b…","""4505ed8e-f087-5ee2-8c67-775daa…","""d2deff68-20a8-5d92-b689-a57381…",null,null
54,0.024942,0.08525,0.049706,3.4834e-8,1,"""branch""",1960-01-01 00:00:00 UTC,2025-01-13 06:58:59.266502 UTC,"""e46fa9df-d633-5149-96b4-c9cccc…","""23bc00b6-0e27-5e6d-a02e-dda5e9…","""d2deff68-20a8-5d92-b689-a57381…",null,null
55,0.024942,0.05425,0.031631,2.2167e-8,1,"""branch""",1960-01-01 00:00:00 UTC,2025-01-13 06:58:59.266502 UTC,"""34edd04e-a827-5cf8-b233-43d285…","""b1d51456-8036-5737-accc-1103d2…","""23bc00b6-0e27-5e6d-a02e-dda5e9…",null,null


In [7]:
branch_parameter_event: pl.DataFrame = branch.with_columns(
    c("uuid").alias("eq_fk"),
    c("uuid").pipe(generate_random_uuid).alias("uuid"),
    c("r_pu").alias("r"),
    c("x_pu").alias("x"),
    c("b_pu").alias("b"),
    c("start").alias("timestamp"),
    pl.lit(heartbeat).alias("heartbeat"),
    pl.lit(SCADA).alias("source_fk"),  # ??? why or why not ?
).with_columns(pl.lit(0.0).alias(col) for col in ["g", "r0", "x0", "b0", "g0"])

new_tables_pl: dict[str, pl.DataFrame] = {
    "Resource": branch,
    "Equipment": branch,
    "Branch": branch,
    "BranchParameterEvent": branch_parameter_event,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="branch"
)
changes_schema = generate_connectivity_table(
    changes_schema=changes_schema, eq_table=branch, raw_data_table="branch"
)

## Energy consumer

In [8]:
default_install_date: datetime = datetime(*settings.DEFAULT_INSTALL_DATE, tzinfo=UTC)


# Power in PU
energy_consumer = df_topology.with_columns(
    pl.lit(ENERGY_CONSUMER).alias("concrete_class"),
    pl.lit(default_install_date).alias("start"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    pl.lit("unknown").alias("profile_type"),
    pl.lit(0).alias("rated_p"),  # Symbol: P_ec_nom, Unit: kW
    c("cn_fk")
    .pipe(generate_random_uuid)
    .alias("uuid"),  # Generate random uuid on a random column
    c("Pload").replace(0, None).alias("node_with_consumer"),
).drop_nulls(
    "node_with_consumer"
)  # Remove node without consumption
new_tables_pl: dict[str, pl.DataFrame] = {
    "Resource": energy_consumer,
    "Equipment": energy_consumer,
    "EnergyConsumer": energy_consumer,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="energy_consumer"
)
changes_schema = generate_connectivity_table(
    changes_schema=changes_schema, eq_table=branch, raw_data_table="energy_consumer"
)

In [9]:
changes_schema.equipment

diff,uuid
str,str
"""+""","""b4fdf239-cfe2-5453-91c3-84c1b2…"
"""+""","""3edce6fc-e164-5b3e-9b0d-8a1c86…"
"""+""","""c173d7f3-f7ca-5f44-98d8-63c1aa…"
"""+""","""c33afba9-0379-5d6a-9ca1-743cf7…"
"""+""","""020bd95c-9320-5560-9627-4bbe19…"
…,…
"""+""","""65363afa-9ba7-4cce-bea0-0abb9a…"
"""+""","""335befee-fca4-46aa-bb14-7914bf…"
"""+""","""8f11961c-be42-4756-9767-5d67c1…"


## Measurement

Active power

In [10]:
## Add the uuid of the node to the power data
measurement = energy_consumer.select(
    c("uuid")
    .pipe(generate_random_uuid)
    .alias(
        "uuid"
    ),  # Generate random uuid on a column without importance (don't work with pl.lit)
    c("uuid").alias("resource_fk"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    pl.lit(MeasurementClass.SPAN.value).alias("concrete_type"),
    pl.lit(MeasurementPhase.ABC.value).alias("phase"),
    pl.lit(MeasurementColumn.DOUBLE.value).alias("column_type"),
    pl.lit(CONVENTIONAL_METER).alias("source_fk"),
    # pl.lit(60*60*24*365).alias("default_period"),
    pl.lit(ACTIVE_POWER).alias("measurement_type"),
    pl.lit("pu").alias("unit_symbol"),
    pl.lit(1).alias("unit_multiplier"),
    c("Pload").alias("double_value"),
)
measurement_span = measurement.with_columns(
    c("uuid").alias("measurement_fk"),
    c("uuid").pipe(generate_random_uuid).alias("uuid"),
    pl.lit(datetime(2022, 1, 1))
    .dt.replace_time_zone(time_zone="Europe/Zurich")
    .dt.convert_time_zone(time_zone="UTC")
    .alias("start"),
    pl.lit(datetime(2023, 1, 1))
    .dt.replace_time_zone(time_zone="Europe/Zurich")
    .dt.convert_time_zone(time_zone="UTC")
    .alias("end"),
)

new_tables_pl: dict[str, pl.DataFrame] = {
    "Measurement": measurement,
    "MeasurementSpan": measurement_span,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="meter_id"
)

Reactive power

In [11]:
## Add the uuid of the node to the power data
measurement = energy_consumer.select(
    c("uuid")
    .pipe(generate_random_uuid)
    .alias(
        "uuid"
    ),  # Generate random uuid on a column without importance (don't work with pl.lit)
    c("uuid").alias("resource_fk"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    pl.lit(MeasurementClass.SPAN.value).alias("concrete_type"),
    pl.lit(MeasurementPhase.ABC.value).alias("phase"),
    pl.lit(MeasurementColumn.DOUBLE.value).alias("column_type"),
    pl.lit(CONVENTIONAL_METER).alias("source_fk"),
    # pl.lit(60*60*24*365).alias("default_period"),
    pl.lit(REACTIVE_POWER).alias("measurement_type"),
    pl.lit("pu").alias("unit_symbol"),
    pl.lit(1).alias("unit_multiplier"),
    c("Qload").alias("double_value"),
)
measurement_span = measurement.with_columns(
    c("uuid").alias("measurement_fk"),
    c("uuid").pipe(generate_random_uuid).alias("uuid"),
    pl.lit(datetime(2022, 1, 1))
    .dt.replace_time_zone(time_zone="Europe/Zurich")
    .dt.convert_time_zone(time_zone="UTC")
    .alias("start"),
    pl.lit(datetime(2023, 1, 1))
    .dt.replace_time_zone(time_zone="Europe/Zurich")
    .dt.convert_time_zone(time_zone="UTC")
    .alias("end"),
)

new_tables_pl: dict[str, pl.DataFrame] = {
    "Measurement": measurement,
    "MeasurementSpan": measurement_span,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="meter_id"
)

## Switch

In [12]:
# Filter to take only switch, connection_type == 3
switch = linedata_distflow.filter(c("connection_type") == 3).select(
    c("line_number").alias("dso_code"),
    pl.lit(SWITCH).alias("concrete_class"),
    pl.lit(default_install_date).alias("start"),
    pl.lit(heartbeat).alias("start_heartbeat"),
    pl.lit(switch_state).alias("normal_open"),
    pl.lit(switch_type).alias("type"),
    pl.lit(switch_command).alias("command"),
    # Generate uuid for each terminal of branch with node uuid
    c("node_from").replace_strict(connectivity_node, default=None).alias("t1"),
    c("node_to").replace_strict(connectivity_node, default=None).alias("t2"),
    # Need column name for validation of the schema
    pl.lit(None).alias("t1_container_fk"),
    pl.lit(None).alias("t2_container_fk"),
    c("line_number").pipe(generate_uuid_col, added_string=SWITCH).alias("uuid"),
)
new_tables_pl: dict[str, pl.DataFrame] = {
    "Resource": switch,
    "Equipment": switch,
    "Switch": switch,
}
changes_schema = add_table_to_changes_schema(
    schema=changes_schema, new_tables_pl=new_tables_pl, raw_table_name="switch"
)
changes_schema = generate_connectivity_table(
    changes_schema=changes_schema, eq_table=switch, raw_data_table="switch"
)

In [13]:
# Begin time of the data from matlab (from main_FC.ipynb before)
str(datetime(2020, 4, 4, 23, 00, 0, 0, UTC) - dt.timedelta(hours=192))

'2020-03-27 23:00:00+00:00'

## Distflow

In [14]:
# changes_schema.connectivity
# changes_schema.measurement["resource_fk"][0]
# changes_schema.branch.filter(c("uuid") == "df941fce-ceda-5874-ab63-5c8af9bec38b")
# changes_schema.connectivity.filter(
#     c("cn_fk").is_in(changes_schema.measurement["resource_fk"])
# )
list(changes_schema.__dict__.keys())

['heartbeat',
 'resource',
 'equipment',
 'terminal',
 'busbar_section',
 'branch',
 'branch_parameter_event',
 'geo_event',
 'switch',
 'switch_event',
 'transformer',
 'transformer_end',
 'transformer_parameter_event',
 'tap',
 'tap_event',
 'bess',
 'energy_consumer',
 'external_network',
 'generating_unit',
 'container',
 'client',
 'substation',
 'base_voltage',
 'connectivity_node',
 'connectivity',
 'measurement',
 'measurement_point',
 'measurement_span']

## Import data to changes schema

In [15]:
# energy_consumer.filter(c("node_number") == 5)
print(branch.filter(c("dso_code") == 5))
print(
    changes_schema.terminal.select(c("eq_fk")).filter(
        c("eq_fk") == "c33afba9-0379-5d6a-9ca1-743cf7c3512d"
    )
)

shape: (1, 14)
┌──────────┬────────────┬────────┬───────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ dso_code ┆ current_li ┆ r_pu   ┆ x_pu      ┆ … ┆ t1         ┆ t2         ┆ t1_contai ┆ t2_contai │
│ ---      ┆ mit        ┆ ---    ┆ ---       ┆   ┆ ---        ┆ ---        ┆ ner_fk    ┆ ner_fk    │
│ i64      ┆ ---        ┆ f64    ┆ f64       ┆   ┆ str        ┆ str        ┆ ---       ┆ ---       │
│          ┆ f64        ┆        ┆           ┆   ┆            ┆            ┆ null      ┆ null      │
╞══════════╪════════════╪════════╪═══════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 5        ┆ 0.024942   ┆ 0.0775 ┆ 0.0451875 ┆ … ┆ dbd2411e-1 ┆ ba84d70a-8 ┆ null      ┆ null      │
│          ┆            ┆        ┆           ┆   ┆ e87-5956-8 ┆ 0d7-590e-b ┆           ┆           │
│          ┆            ┆        ┆           ┆   ┆ 6d9-d69ee7 ┆ 112-f9c4b5 ┆           ┆           │
│          ┆            ┆        ┆           ┆   ┆ …          ┆ …          ┆

In [16]:
changes_schema.terminal.select(c("eq_fk")).filter(
    c("eq_fk") == "c33afba9-0379-5d6a-9ca1-743cf7c3512d"
)
changes_schema.connectivity.with_columns(
    c("cn_fk").is_unique().alias("unique")
)  # Give true for the slack node...
test2 = changes_schema.connectivity.filter(
    c("cn_fk").is_first_distinct() == True
)  # Give true for the slack node...

In [17]:
v_slack_sq = pow(parameter_distflow["Vslack"][0], 2)
test = changes_schema.connectivity  # .filter(c("side") == "t1")
# line_data
# "downstream": [1, 2, 3, 4, 5, 6, 7, 8],
# "upstream": [None, 1, 2, 1, 4, 4, 4, 6],
# "P": [0, 1, 2, 1, 4, 3, 6, 5],
# "F": [0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
# "p_line": [0] * 8,

In [18]:
dist_flow_df : pl.DataFrame = 
iteration = 0
# parameter_distflow
# ((norm(Vnode_sq(:,2) - Vnode_sq(:,1)) > tol  && iteration < maxIteration))
tol = parameter_distflow["tol"][0]
max_iteration = parameter_distflow["maxIteration"][0]


x = 
while x < tol and iteration < max_iteration:
    print(iteration)
    iteration += 1
# Qload_augmented = Qload - Bnode .* Vnode_sq(:,1);

SyntaxError: invalid syntax (1459228918.py, line 1)

In [ ]:
def sum_downstream_power(col: pl.Expr, df: pl.DataFrame):
    return col.map_elements(
        lambda x: df.filter(c("upstream") == x)["p_line"].sum(), return_dtype=pl.Float64
    )


def calculate_line_power(df: pl.DataFrame):
    return (c("downstream").pipe(sum_downstream_power, df=df) + c("P")) * (1 + c("F"))


def sum_power(df: pl.DataFrame, lv: int):

    return df.with_columns(
        pl.when(c("lv") == lv)
        .then(calculate_line_power(df=df))
        .otherwise(c("p_line"))
        .alias("p_line")
    )


# UP Use for each powerflow
# Down Use only one time
def get_node_level(G: nx.DiGraph) -> dict:
    level_mapping: dict = {}
    for node in reversed(list(nx.topological_sort(G))):
        if not len(list(G.successors(node))):
            level_mapping[node] = 0
        else:
            level_mapping[node] = max(level_mapping[n] for n in G.successors(node)) + 1
    return level_mapping


line_data: pl.DataFrame = pl.DataFrame(
    {
        "downstream": [1, 2, 3, 4, 5, 6, 7, 8],
        "upstream": [None, 1, 2, 1, 4, 4, 4, 6],
        "P": [0, 1, 2, 1, 4, 3, 6, 5],
        "F": [0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
        "p_line": [0] * 8,
    }
)

grid = nx.DiGraph()

_ = line_data.drop_nulls(subset="upstream").with_columns(
    pl.struct(c("upstream"), c("downstream")).map_elements(
        lambda x: grid.add_edge(x["upstream"], x["downstream"]), return_dtype=pl.Struct
    )
)
level_mapping: dict = get_node_level(G=grid)
line_data = line_data.with_columns(
    c("downstream").replace_strict(level_mapping, default=None).alias("lv")
)

for i in range(line_data["lv"].max() + 1):
    line_data = sum_power(df=line_data, lv=i)

print(line_data.sort("lv"))

shape: (8, 6)
┌────────────┬──────────┬─────┬─────┬────────┬─────┐
│ downstream ┆ upstream ┆ P   ┆ F   ┆ p_line ┆ lv  │
│ ---        ┆ ---      ┆ --- ┆ --- ┆ ---    ┆ --- │
│ i64        ┆ i64      ┆ i64 ┆ f64 ┆ f64    ┆ i64 │
╞════════════╪══════════╪═════╪═════╪════════╪═════╡
│ 3          ┆ 2        ┆ 2   ┆ 0.1 ┆ 2.2    ┆ 0   │
│ 5          ┆ 4        ┆ 4   ┆ 0.1 ┆ 4.4    ┆ 0   │
│ 7          ┆ 4        ┆ 6   ┆ 0.1 ┆ 6.6    ┆ 0   │
│ 8          ┆ 6        ┆ 5   ┆ 0.1 ┆ 5.5    ┆ 0   │
│ 2          ┆ 1        ┆ 1   ┆ 0.1 ┆ 3.52   ┆ 1   │
│ 6          ┆ 4        ┆ 3   ┆ 0.1 ┆ 9.35   ┆ 1   │
│ 4          ┆ 1        ┆ 1   ┆ 0.1 ┆ 23.485 ┆ 2   │
│ 1          ┆ null     ┆ 0   ┆ 0.0 ┆ 27.005 ┆ 3   │
└────────────┴──────────┴─────┴─────┴────────┴─────┘
